<a href="https://colab.research.google.com/github/mneedham/data-science-training/blob/master/03_Recommendations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Recommendations: Part 2

In the 2nd part of our recommendations notebook, we're going to use the PageRank algorithm to make article recommendations to an author. Let's import our libraries in case we don't have those from the previous notebooks:

In [3]:
from py2neo import Graph
import pandas as pd

import matplotlib 
import matplotlib.pyplot as plt

plt.style.use('fivethirtyeight')
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option('display.max_colwidth', 100)

## PageRank

We're going to use the PageRank algorithm, so let's first get up to speed on this algorithm. 

PageRank is an algorithm that measures the transitive influence or connectivity of nodes. It can be computed by either iteratively distributing one node’s rank (originally based on degree) over its neighbours or by randomly traversing the graph and counting the frequency of hitting each node during these walks.

## Full Text Search

The following code will create a full text search index on the 'title' and 'abstract' properties of all nodes that have the label 'Article'

In [ ]:
query = """
CALL db.index.fulltext.createNodeIndex('articles', ['Article'], ['title', 'abstract'])
"""
graph.run(query).to_data_frame()

## Article recommendation

When an author is searcing for articles to read, they want that search to take them into account. Two authors using the same search term would expect to see different results depending on their area of research.

We're going to use the Full Text Search functionality added in Neo4j 3.5 to help us with the search part of the problem.

The following

In [37]:
query = """
MATCH (a:Article)-[:AUTHOR]->(author:Author)
WHERE author.name=$authorName
WITH author, collect(a) as articles
CALL algo.pageRank.stream(
  'CALL db.index.fulltext.queryNodes("articles", $searchTerm)
   YIELD node, score
   RETURN id(node) as id',
  'MATCH (a1:Article)-[:CITED]->(a2:Article) 
   RETURN id(a1) as source,id(a2) as target', 
  {sourceNodes: articles,graph:'cypher', params: {searchTerm: $searchTerm}})
YIELD nodeId, score
WITH algo.getNodeById(nodeId) AS n, score
WHERE not(exists((author)<-[:AUTHOR]-(n)))
RETURN n.title as article, score, [(n)-[:AUTHOR]->(author) | author.name][..5] AS authors
order by score desc limit 10
"""

params = {"authorName": "Tao Xie", "searchTerm": "open source"}
graph.run(query, params).to_data_frame()

article  \
0                             Using structural context to recommend source code examples   
1                               Static detection of cross-site scripting vulnerabilities   
2                                                        Evolutionary testing of classes   
3                                            Clone detection using abstract syntax trees   
4                          Bandera: extracting finite-state models from Java source code   
5  Concern graphs: finding and describing concerns using structural program dependencies   
6                                                         The source code control system   
7                                                       Detecting object usage anomalies   
8                          Generalized symbolic execution for model checking and testing   
9                         Hipikat: recommending pertinent software development artifacts   

                                                                                         authors  \
0                                                                  [Reid Holmes, Gail C. Murphy]   
1                                                                 [Gary Wassermann, Zhendong Su]   
2                                                                                [Paolo Tonella]   
3  [Lorraine Bier, Marcelo M. SantAnna, Leonardo Mendonça de Moura, Andrew Yahin, Ira D. Baxter]   
4                      [Hongjun Zheng, Robby, Corina S. Pasareanu, Shawn Laubach, John Hatcliff]   
5                                                          [Martin P. Robillard, Gail C. Murphy]   
6                                                                             [Marc J. Rochkind]   
7                                        [Christian Lindig, Andrzej Wasylkowski, Andreas Zeller]   
8                                         [Sarfraz Khurshid, Corina S. Păsăreanu, Willem Visser]   
9                                                               [Davor Cubranic, Gail C. Murphy]   

   score  
0  0.680  
1  0.382  
2  0.314  
3  0.308  
4  0.299  
5  0.276  
6  0.256  
7  0.246  
8  0.246  
9  0.226

In [36]:
params = {"authorName": "Margus Veanes", "searchTerm": "open source"}
graph.run(query, params).to_data_frame()

article  \
0                                                      The source code control system   
1                              Program Improvement by Source-to-Source Transformation   
2                                  Make — a program for maintaining computer programs   
3            Two case studies of open source software development: Apache and Mozilla   
4                             Improving and refining programs by program manipulation   
5                                              Equivariant adaptive source separation   
6  StackGuard: automatic adaptive detection and prevention of buffer-overflow attacks   
7                                         Clone detection using abstract syntax trees   
8                                A New Learning Algorithm for Blind Signal Separation   
9                                                   Building diverse computer systems   

                                                                                         authors  \
0                                                                             [Marc J. Rochkind]   
1                                                                             [David B. Loveman]   
2                                                                            [Stuart I. Feldman]   
3                                               [Audris Mockus, Roy Fielding, James D. Herbsleb]   
4                                  [Dennis F. Kibler, James Milne Neighbors, Thomas A. Standish]   
5                                                             [Beate Hvam Laheld, J.-F. Cardoso]   
6                                [Qian Zhang, Perry Wagle, Aaron Grier, Steve Beattie, P. Bakke]   
7  [Lorraine Bier, Marcelo M. SantAnna, Leonardo Mendonça de Moura, Andrew Yahin, Ira D. Baxter]   
8                                           [Shun-ichi Amari, Andrzej Cichocki, Howard Hua Yang]   
9                                            [Stephanie Forrest, Anil Somayaji, David H. Ackley]   

   score  
0 22.130  
1 16.746  
2 16.325  
3 15.716  
4 15.708  
5 15.261  
6 10.425  
7 10.250  
8  9.989  
9  9.729